In [12]:
import pandas as pd
import pyodbc as pyo
import numpy as np
import os

In [13]:
pyo.drivers()

['SQL Server',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [14]:
def connect_database():
    drivers = 'ODBC Driver 17 for SQL Server'
    SeverName = 'DESKTOP-L8OJDR9\SQLEXPRESS01'
    db = 'CHUNGKHOAN'
    connection = pyo.connect(driver = drivers, 
                            host = SeverName ,database = db ,trusted_connection = 'yes')
    conn = connection
    cur = conn.cursor()
    return cur,conn
def create_table(cur,conn,query):
    cur.execute(query)
    conn.commit()
def insert_table(cur,conn,query,df1):
    for i,row in df1.iterrows():
        cur.execute(query,list(row))
    conn.commit()

In [15]:
cur,conn = connect_database()

In [16]:
query_create_table = ("""CREATE TABLE DATA_MAJOR (
                            STT int IDENTITY,
                            Stock_code varchar(20),
                            Name_Company nvarchar(100),
                            Industry nvarchar(100),
                            Exchange nvarchar(10),
                            KL_CPNY_CP float,
                            Majority nvarchar(100) 
                            ) ;
                          CREATE TABLE COMPANY (
                            idx int NOT NULL,
                            Company_Code varchar(20) PRIMARY KEY ,
                            Full_Name nvarchar(100),
                            Company_ID int,
                            Cat_ID int );
                           """)

In [17]:
query_insert_table_1 = ("""INSERT INTO DATA_MAJOR (
                            Stock_code,
                            Name_Company,
                            Industry,
                            Exchange, 
                            KL_CPNY_CP,
                            Majority ) VALUES (?,?,?,?,?,?)""")
query_insert_table_2 = ("""INSERT INTO COMPANY ( 
                                idx, 
                                Company_Code,
                                Full_Name,
                                Company_ID,
                                Cat_ID ) VALUES (?,?,?,?,?)""")

In [18]:
df1 = pd.read_csv('D:\chungkhoan\data.csv')
df1.head(5)

,Number,Stock_code,Name_Company,Industry,Exchange,KL_CPNY_(CP),Majority
0,1,AAT,CTCP Tập Đoàn Tiên Sơn Thanh Hóa,Bán buôn hàng tiêu dùng,HOSE,63.801.489,Bán buôn
1,2,AAV,CTCP AAV Group,Bán buôn hàng tiêu dùng,HNX,68.987.661,Bán buôn
2,3,ABR,CTCP Đầu tư Nhãn hiệu Việt,Bán buôn hàng tiêu dùng,HOSE,20.000.000,Bán buôn
3,4,ABS,CTCP Dịch vụ Nông nghiệp Bình Thuận,Bán buôn hàng tiêu dùng,HOSE,80.000.000,Bán buôn
4,5,AMP,CTCP Armephaco,Bán buôn hàng lâu bền,UpCom,13.000.000,Bán buôn


In [19]:
df1['KL_CPNY_(CP)'] = df1['KL_CPNY_(CP)'].apply(lambda x: x.replace('.',''))
df1['KL_CPNY_(CP)'] = df1['KL_CPNY_(CP)'].apply(lambda x: int(x))

In [20]:
df1.columns

Index(['Number', 'Stock_code', 'Name_Company', 'Industry', 'Exchange',
       'KL_CPNY_(CP)', 'Majority'],
      dtype='object')

In [21]:
column = ['Stock_code', 'Name_Company', 'Industry', 'Exchange',
       'KL_CPNY_(CP)', 'Majority']
df1 = df1[column]
df1.head(5)

,Stock_code,Name_Company,Industry,Exchange,KL_CPNY_(CP),Majority
0,AAT,CTCP Tập Đoàn Tiên Sơn Thanh Hóa,Bán buôn hàng tiêu dùng,HOSE,63801489,Bán buôn
1,AAV,CTCP AAV Group,Bán buôn hàng tiêu dùng,HNX,68987661,Bán buôn
2,ABR,CTCP Đầu tư Nhãn hiệu Việt,Bán buôn hàng tiêu dùng,HOSE,20000000,Bán buôn
3,ABS,CTCP Dịch vụ Nông nghiệp Bình Thuận,Bán buôn hàng tiêu dùng,HOSE,80000000,Bán buôn
4,AMP,CTCP Armephaco,Bán buôn hàng lâu bền,UpCom,13000000,Bán buôn


In [22]:
create_table(cur,conn,query_create_table)
insert_table(cur,conn,query_insert_table_1,df1)

In [23]:
df2 = pd.read_csv('D:\chungkhoan\Company.csv')
insert_table(cur,conn,query_insert_table_2,df2)

In [24]:
cur.close()
conn.close()